In [3]:
def compare_date_components(day_doc, month_doc, year_doc, day_req, month_req, year_req):
    """Compare two date components.

    Args:
        day_doc (int): Day of the document.
        month_doc (int): Month of the document.
        year_doc (int): Year of the document.
        day_req (int): Day of the request.
        month_req (int): Month of the request.
        year_req (int): Year of the request.

    Returns:
        int: -1 if date_doc < date_requete, 0 if equal, 1 if date_doc > date_requete.
    """
    if year_doc < year_req:
        return -1
    elif year_doc > year_req:
        return 1
    else:
        if month_doc < month_req:
            return -1
        elif month_doc > month_req:
            return 1
        else:
            if day_doc < day_req:
                return -1
            elif day_doc > day_req:
                return 1
            else:
                return 0
def compare_dates(date_doc, date_requete):
    """ date_requete = resultat = {"début": None, "fin": None, "précis": None, "not": None}
    avec value de chaque key = {"j": None, "m": None, "a": None} 
    date_doc = 'j/m/a'
    Args:
        date1 (str): The date of the doucument to compare; format 'j/m/a'.
        date2 (str): The date de la requête à comparer; format {"début": None, "fin": None, "précis": None, "not": None}
            with each value being a dictionary: {"j": None, "m": None, "a": None}.

    Returns:
        True if date_doc fits date_requete, False otherwise.
    """
    # Split the dates into components
    day_doc, month_doc, year_doc = date_doc.split('/')

    #check date_requete empty aka no restriction aka always True
    if date_requete["début"] is None and date_requete["fin"] is None and date_requete["précis"] is None and date_requete["not"] is None:
        return True

    #check date_requete["not"]
    if date_requete["not"] is not None: 
        day_not, month_not, year_not = int(date_requete["not"]["j"]), int(date_requete["not"]["m"]), int(date_requete["not"]["a"])
        if day_doc == day_not and month_doc == month_not and year_doc == year_not:
            return False

    #compare date_requete["précis"]
    if date_requete["précis"] is not None:
        day_req, month_req, year_req = int(date_requete["précis"]["j"]), int(date_requete["précis"]["m"]), int(date_requete["précis"]["a"])
        if day_doc == day_req and month_doc == month_req and year_doc == year_req:
            return True
        else:
            return False
        
    res = True
    # Compare date_requete["début"]
    if date_requete["début"] is not None:
        day_req, month_req, year_req = int(date_requete["début"]["j"]), int(date_requete["début"]["m"]), int(date_requete["début"]["a"])
        if compare_date_components(day_doc, month_doc, year_doc, day_req, month_req, year_req) < 0:
            res = res and False
    
    # Compare date_requete["fin"]
    if date_requete["fin"] is not None:
        day_req, month_req, year_req = int(date_requete["fin"]["j"]), int(date_requete["fin"]["m"]), int(date_requete["fin"]["a"])
        if compare_date_components(day_doc, month_doc, year_doc, day_req, month_req, year_req) > 0:
            res = res and False
    
    # Dates are equal
    return res

In [4]:
def compare_date_components(day_doc, month_doc, year_doc, day_req, month_req, year_req):
    """Compare two date components, handling None values in the request."""
    # Convert all to int for doc, but req can be None
    if year_req is not None:
        if int(year_doc) < int(year_req):
            return -1
        elif int(year_doc) > int(year_req):
            return 1
    if month_req is not None:
        if int(month_doc) < int(month_req):
            return -1
        elif int(month_doc) > int(month_req):
            return 1
    if day_req is not None:
        if int(day_doc) < int(day_req):
            return -1
        elif int(day_doc) > int(day_req):
            return 1
    return 0

def compare_dates(date_doc, date_requete):
    """
    Compare une date de document à une requête de date, en tenant compte des champs manquants (None).
    """
    # Split the document date into components
    day_doc, month_doc, year_doc = date_doc.split('/')
    day_doc, month_doc, year_doc = int(day_doc), int(month_doc), int(year_doc)

    # If no restriction, always True
    if all(date_requete.get(k) is None for k in ["début", "fin", "précis", "not"]):
        return True

    # Handle "not"
    not_req = date_requete.get("not")
    if not_req is not None:
        jn, mn, an = not_req.get("j"), not_req.get("m"), not_req.get("a")
        match = True
        if an is not None and int(year_doc) != int(an):
            match = False
        if mn is not None and int(month_doc) != int(mn):
            match = False
        if jn is not None and int(day_doc) != int(jn):
            match = False
        if match:
            return False

    # Handle "précis"
    precis = date_requete.get("précis")
    if precis is not None:
        jp, mp, ap = precis.get("j"), precis.get("m"), precis.get("a")
        match = True
        if ap is not None and int(year_doc) != int(ap):
            match = False
        if mp is not None and int(month_doc) != int(mp):
            match = False
        if jp is not None and int(day_doc) != int(jp):
            match = False
        return match

    res = True
    # Handle "début"
    debut = date_requete.get("début")
    if debut is not None:
        jd, md, ad = debut.get("j"), debut.get("m"), debut.get("a")
        cmp = compare_date_components(day_doc, month_doc, year_doc,
                                      int(jd) if jd is not None else None,
                                      int(md) if md is not None else None,
                                      int(ad) if ad is not None else None)
        if cmp < 0:
            res = False

    # Handle "fin"
    fin = date_requete.get("fin")
    if fin is not None:
        jf, mf, af = fin.get("j"), fin.get("m"), fin.get("a")
        cmp = compare_date_components(day_doc, month_doc, year_doc,
                                      int(jf) if jf is not None else None,
                                      int(mf) if mf is not None else None,
                                      int(af) if af is not None else None)
        if cmp > 0:
            res = False

    return res

In [117]:
import pandas as pd
import ast
#rubrique et dates dans others.csv
# mots_clés dans text_title.csv

'''
    resultats = {
        "return": None,
        "mots_cles": {"yes": [], "no": None},
        "operateurs_mots_cles": None,
        "rubrique": None,
        "operateurs_rubrique": None,
        "dates": {"debut": None, "fin": None, "précis": None, "not": None},
        "titre": None,
        "operateurs_titre": None,
        "images": None
    }

'''
def recherche_documents(resultats, index_inverse_texte, index_inverse_date, index_inverse_rubrique, index_inverse_titre, index_inverse_image):
    """
    Fonction qui recherche les documents correspondant, dans lemmes_path, aux résultats de la requete (resultats).
    :param resultats: dictonnaires contenant les résultats de la requête
    :param index_inverse_mots_cles: fichier csv index inversé des mots-clés
    :param index_inverse_dates_rubriques: fichier csv index inversé des dates et rubriques
    :return: Liste de documents correspondant aux résultats de la requête
    """
    docs_cherches = []
    #partie mots-clés (sans les not)
    mots_cles = resultats["mots_cles"]
    if len(mots_cles["yes"]) != 0:
        dict_match_key = {}
        index_texte = pd.read_csv(index_inverse_texte, sep= "\t")
        for mot in mots_cles["yes"]:
            docs_qui_matchs = index_texte.loc[index_texte["mot"] == mot, "docs"].values[0]
            dict_match_key[mot] = docs_qui_matchs
        if resultats["operateurs_mots_cles"] == "ou":
            docs_key = list(set.union(*(set(ast.literal_eval(l)) for l in dict_match_key.values())))
        else:
            docs_key = list(set.intersection(*(set(ast.literal_eval(l)) for l in dict_match_key.values())))

        if len(docs_cherches) == 0:
            docs_cherches = docs_key
        else:
            docs_cherches = list(set(docs_cherches) & set(docs_key))

    #partie titre
    titres = resultats["titre"]
    if titres != None:
        dict_match_titre = {}
        index_titre = pd.read_csv(index_inverse_titre, sep='\t')
        if isinstance(titres) != list:
            titres = [].append(titres)
        for titre in titres:
            docs_qui_matchs = index_titre.loc[index_titre["mot"] == titre, "docs"].values[0]
            dict_match_titre[titre] = docs_qui_matchs
        if resultats["operateurs_titre"] == "ou":
            docs_titre = list(set().union(*(set(ast.literal_eval(l)) for l in list(dict_match_titre.values()))))
        else:
            docs_titre = list(set.intersection(*(set(ast.literal_eval(l)) for l in list(dict_match_titre.values()))))
        
        if len(docs_cherches) == 0:
            docs_cherches = docs_titre
        else:
            docs_cherches = list(set(docs_cherches) & set(docs_titre))
    
    #partie rubrique
    rubriques = resultats["rubrique"]
    if rubriques != None:
        print(rubriques)
        dict_match_rubrique = {}
        index_rubrique = pd.read_csv(index_inverse_rubrique, sep='\t')
        if not isinstance(rubriques, list):
            liste = [rubriques]
            rubriques = liste
        for rubrique in rubriques:
            docs_matchs = index_rubrique.loc[index_rubrique["mot"] == rubrique, "docs"].values
            if len(docs_matchs) > 0:
                docs_qui_matchs = docs_matchs[0]
                dict_match_rubrique[rubrique] = docs_qui_matchs
            else:
                print(f"⚠️ Rubrique introuvable : {rubrique}")
            
            dict_match_rubrique[rubrique] = docs_qui_matchs
        if resultats["operateurs_titre"] == "ou":
            docs_titre = list(set().union(*(set(ast.literal_eval(l)) for l in list(dict_match_rubrique.values()))))
        else:
            docs_titre = list(set.intersection(*(set(ast.literal_eval(l)) for l in list(dict_match_rubrique.values()))))
        
        if len(docs_cherches) == 0:
            docs_cherches = docs_titre
        else:
            docs_cherches = list(set(docs_cherches) & set(docs_titre))

    #partie images
    image = resultats["images"]
    if image != None:
        index_image = pd.read_csv(index_inverse_image, sep='\t')
        docs_match_image = index_image.loc[index_rubrique["mot"] == "yes", "docs"].values[0]
        if len(docs_cherches) == 0:
            docs_cherches = docs_titre
        else:
            docs_cherches = list(set(docs_cherches) & set(docs_match_image))

    #partie date
    date = resultats["dates"]
    if date["début"] != None or date["fin"] != None or date["précis"] != None or date["not"] != None:
        index_date = pd.read_csv(index_inverse_date, sep='\t')
        for date_doc_comp in index_date:
            print("dates ____________________")
            print(date_doc_comp)
            print(date)
            if compare_dates(date_doc=date_doc_comp, date_requete= date):
                docs_qui_matchs = date_doc_comp["docs"]
                if len(docs_cherches) == 0:
                    docs_cherches = docs_titre
                else:
                    docs_cherches = list(set(docs_cherches) & set(docs_match_image))

    #partie not
    mot_cles_not = mots_cles["no"]
    if mot_cles_not != None:
        index_texte = pd.read_csv(index_inverse_texte, sep= "\t")
        docs_qui_matchs = index_texte.loc[index_texte["mot"] == mot_cles_not, "docs"].values[0]
        
        docs_cherches = list(set(docs_cherches) - set(docs_qui_matchs))


    #partie return_value
    if resultats["return"] != "rubriques":
        return docs_cherches
    else:
        rubriques_cherches = []
        index_rubrique = pd.read_csv(index_inverse_rubrique, sep='\t')
        for rubrique in index_rubrique:
            for doc in docs_cherches:
                if doc in rubrique["docs"]:
                    rubriques_cherches.append(rubrique["docs"])
        return rubriques_cherches

In [118]:
from td5 import *
from td6 import *

if __name__ == "__main__":
    # extraction des informations de la requete
    requete = input("Entrez votre requête en langage naturel : ")
    resultat = traiter_requete(requete)

    # correction orthographique des mots clés
    lemmes_path = "lemmes.csv"
    if resultat["mots_cles"]["no"] != None:
        resultat["mots_cles"]["no"] = correction_orthographique(resultat["mots_cles"]["no"], lemmes_path)
    
    for i in range(len(resultat["mots_cles"]["yes"])):
        resultat["mots_cles"]["yes"][i] = correction_orthographique(resultat["mots_cles"]["yes"][i], lemmes_path)
    
    print("Requête corrigée :")
    print(resultat)
    print("-------------------------------------------")

    index_inverse_texte = "../TD4/reverse_index_texte.csv"
    index_inverse_date = "../TD4/reverse_index_date.csv"
    index_inverse_rubrique = "../TD4/reverse_index_rubrique.csv"
    index_inverse_titre = "../TD4/reverse_index_titre.csv"
    index_inverse_image = "../TD4/reverse_index_image.csv"

    # recherche des documents pertinents
    documents = recherche_documents(resultat, index_inverse_texte, index_inverse_date, index_inverse_rubrique, index_inverse_titre, index_inverse_image)
    print(documents)


no lemma found for:   
Requête corrigée :
{'return': 'articles', 'mots_cles': {'yes': ['travaux'], 'no': None}, 'operateurs_mots_cles': None, 'rubrique': 'actualités innovations', 'operateurs_rubriquedates': {'debut': None, 'fin': None, 'précis': None, 'not': None}, 'titre': None, 'operateurs_titre': None, 'images': None, 'dates': {'début': None, 'fin': None, 'précis': {'j': '1', 'm': '12', 'a': '2012'}, 'not': None}}
-------------------------------------------
actualités innovations
⚠️ Rubrique introuvable : actualités innovations
dates ____________________
mot
{'début': None, 'fin': None, 'précis': {'j': '1', 'm': '12', 'a': '2012'}, 'not': None}


ValueError: not enough values to unpack (expected 3, got 1)